In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline

In [ ]:
all_events=pd.read_csv('raw_data/partial_events.csv')
all_items=pd.read_csv('raw_data/tianchi_fresh_comp_train_item.csv')

all_events['time']=pd.DatetimeIndex(all_events.time)
target_items=set(all_items.item_id)
last_time=all_events.time.sort_values().iloc[-1]
last_day=pd.Timestamp(year=last_time.year,month=last_time.month,day=last_time.day)

last_day_purchase=all_events[np.logical_and(all_events.behavior_type==4,all_events.time>=last_day)]
val_data=last_day_purchase[last_day_purchase.item_id.apply(lambda i: i in target_items)][['user_id','item_id']]
len(last_day_purchase),len(val_data)

In [ ]:
def score(y_true,y_pred):
    true_vals=set((y_true.user_id.apply(lambda u:str(u))+'_'+y_true.item_id.apply(lambda i:str(i))).unique())
    pred_vals=set((y_pred.user_id.apply(lambda u:str(u))+'_'+y_pred.item_id.apply(lambda i:str(i))).unique())
    hits=len(list(filter(lambda x:x in true_vals,pred_vals)))
    prec=hits/len(y_pred)
    rec=hits/len(y_true)
    return 2*prec*rec/(prec+rec),prec,rec

In [ ]:
beg_day=last_day-pd.Timedelta(1,'d')
train_events=all_events[np.logical_and(all_events.time>=beg_day,all_events.time<last_day)]
bought_items=train_events[train_events.behavior_type==4].item_id.unique()
train_events=train_events[train_events.item_id.apply(lambda e:e not in bought_items)]
carted=train_events[train_events.behavior_type==3]
carted=carted[carted.item_id.apply(lambda x:x in target_items)]
score(val_data,carted)

In [ ]:
day_before_last_day=last_day-pd.Timedelta(3,'d')
last_day_evts=all_events[np.logical_and(all_events.time<last_day,all_events.time>day_before_last_day)]
bought_items=last_day_evts[last_day_evts.behavior_type==4].item_id.unique()
last_br_evts=last_day_evts[last_day_evts.behavior_type==1]
last_br_evts=last_br_evts[last_br_evts.item_id.apply(lambda iid:iid in target_items and iid not in bought_items)]
last_br=[]
tmp=set(['a'])
for i in last_br_evts.index:
    uid=last_br_evts.loc[i,'user_id']
    iid=last_br_evts.loc[i,'item_id']
    k=str(uid)+'-'+str(iid)
    c=len(last_br_evts[np.logical_and(
        last_br_evts.user_id==uid,last_br_evts.item_id==iid)])
    if k not in tmp and c>1:
        last_br.append({'user_id':uid,'item_id':iid,'cnt':c})
        tmp.add(k)
last_br=pd.DataFrame(last_br)

score(val_data,last_br)

In [ ]:
last_most_br=last_br.sort_values('cnt',ascending=False)[:len(last_br)//2]
score(val_data,last_most_br)

In [ ]:
day_before_last_day=last_day-pd.Timedelta(3,'d')
last_day_evts=all_events[np.logical_and(all_events.time<last_day,all_events.time>day_before_last_day)]
last_br_evts=last_day_evts
last_br=[]
tmp=set(['a'])
for i in last_br_evts.index:
    uid=last_br_evts.loc[i,'user_id']
    iid=last_br_evts.loc[i,'item_id']
    k=(uid,iid)
    if k not in tmp:
        last_br.append({'user_id':uid,'item_id':iid})
        tmp.add(k)
last_br=pd.DataFrame(last_br)

score(val_data,last_br),score(last_day_purchase,last_br)